In [1]:
import timeit
import json 
import copy
import pickle
import os
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from matplotlib.path import Path
from torchvision import transforms
from torch.utils.data import Dataset
from skimage import draw
from skimage import io
from skimage import transform
import torch
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
plt.rcParams["figure.figsize"] = (15,8)
from skimage.io import imread, imshow, imsave
from skimage.color import rgb2gray
from skimage.draw import polygon
from sklearn.metrics import confusion_matrix
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
sns.set()

import numpy as np
import torch
from torchvision import transforms
image_size = 224


def overlap(a, b, method='dice'):
    a = a.astype(int).flatten()
    b = b.astype(int).flatten()
    tn, fp, fn, tp = confusion_matrix(a, b, labels=[0, 1]).ravel()
    if method == 'dice':
        return 2*tp / (2*tp + fp + fn)
    elif method == 'jaccard':
        return tp / (tp + fp + fn)


def polygon2mask(polygon):
    """
    Create an image mask from polygon coordinates
    """
    vertex_row_coords, vertex_col_coords, shape = polygon[:, 1], polygon[:, 0], (450, 600)
    
    fill_row_coords, fill_col_coords = draw.polygon(vertex_row_coords, vertex_col_coords, shape)
    mask = np.zeros(shape, dtype=float)
    mask[fill_row_coords, fill_col_coords] = 1.
    mask = transform.resize(mask, (image_size, image_size))
    return mask.astype(np.int16)


def process_annotations(y_annotations):
    masks = [polygon2mask(ann) for ann in y_annotations]
    mask = np.bitwise_or.reduce(masks)
    return mask


def plot(polygons, image_id, feature, gt):
    
    fig, axs = plt.subplots(1, 3)

    for i, polygon in enumerate(polygons):
        polygons[i] = process_annotations(polygon)

    img = io.imread("/home/kti01/Documents/My Files/Projects/Overlap/characteristics_classifier/data/HAM10000/HAM10000/"+image_id+'.jpg')
    transform = transforms.Compose([transforms.ToPILImage(),
                                    transforms.Resize((image_size, image_size)),
                                    transforms.ToTensor()])
    image = transform(img)

    axs[0].imshow(torch.permute(image, (1, 2, 0)), alpha=1)
    axs[0].imshow(polygons[0][0], alpha=0.3)
    axs[1].imshow(torch.permute(image, (1, 2, 0)), alpha=1)
    axs[1].imshow(polygons[1][0], alpha=0.3)
    axs[2].imshow(torch.permute(image, (1, 2, 0)), alpha=1)
    axs[2].imshow(cv2.bitwise_and(polygons[0][0].numpy(), polygons[1][0].numpy()), alpha=0.3)
                      
    axs[0].set_xticks([])
    axs[0].set_yticks([])
    axs[1].set_xticks([])
    axs[1].set_yticks([])
    axs[2].set_xticks([])
    axs[2].set_yticks([])
    axs[0].title.set_text('Annotator 1')
    axs[1].title.set_text('Annotator '+gt)
    axs[2].title.set_text('Combined')
    fig.tight_layout()
    fig.suptitle(feature, fontsize="x-large")
    plt.grid(False)
    plt.savefig(os.path.join('combine_annotations/intersection', gt, str(image_id)+'_'+feature))
    plt.close(fig)


char_class_labels = ['TRBL', 'ESA', 'BDG', 'GP', 'PV', 'PRL', 'WLSA', 'PLR', 'PES', 'PIF', 'OPC', 'SPC', 'MVP', 'PRLC', 'PLF', 'PDES', 'APC', 'MS']
#char_class_labels = ['image_id', 'TRBL', 'ESA', 'BDG', 'GP', 'PV', 'PRL', 'WLSA', 'PLR', 'PES', 'PIF', 'OPC', 'SPC', 'MVP', 'PRLC', 'PLF', 'PDES', 'APC', 'MS']
mel_class_labels = ['TRBL', 'ESA', 'BDG', 'GP', 'PV', 'PRL', 'WLSA', 'PES', 'PLR']
nv_class_labels = ['OPC', 'SPC', 'MVP', 'PRLC', 'PLF', 'PDES', 'APC', 'MS']
mel_ann_labels = [label+'_annotation' for label in mel_class_labels]
nv_ann_labels = [label+'_annotation' for label in nv_class_labels]
char_ann_labels = [label+'_annotation' for label in char_class_labels]

/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time
"""
For each image with more than 1 annotator, intersect the annotations.
After intersecting, compare each individual polygon with the intersected annotation.
For each individual polygon, calculate the percentage of information lost (defined by percent of pixels lost).
If this 'percentage pixel lost' value is greater than 0.8, 
 add the individual polygon to the final intersected annotation.
"""

image_dir = "/home/kti01/Documents/My Files/Data/HAM10000/HAM10000/"
metadata = pd.read_pickle("/home/kti01/Documents/My Files/Projects/Overlap/data/ground_truth/metadata_gt_consolidated.pkl")
annotations = {}

# Loop through each image and each feature
for image_id in metadata['image_id'].unique():
    
    # Add the image_id to the dict for the first time
    annotations[image_id] = {}# {k: -1 for k in char_class_labels}
    
    df = metadata[metadata['image_id'] == image_id]
    
    for feature in char_class_labels:

        masks = []
        polygons = []

        for annotation in df[feature+'_annotation'].values:
            if annotation != -1:
                # Store the masks for each individual annotator in masks list and store each individual
                #  polygon in the polygons list
                polygons.extend(annotation)
                masks.append(process_annotations(annotation))

        # If no masks have been added to the list (the feature hasn't been chosen by any annotator),
        #  then skip the current feature.
        if len(masks) == 0:
            continue
        
        # Get the candidate mask by intersecting the individual masks using bitwise_and.reduce().
        # If masks list only has 1 mask, bitwise_and.reduce() will simply remove the outer list,
        #  producing a 2d mask. It is equivalent to masks[0]
        candidate_mask = np.bitwise_and.reduce(np.stack(masks))
        final_mask = candidate_mask.copy()
        
        # If image has been annotated by more than 1 annotator, they need to be combined.
        if len(masks) > 1:
            # Loop through each individual polygon
            for i, polygon in enumerate(polygons):
                # Get the mask for each polygon
                mask = polygon2mask(polygon)

                # Compare the mask with the candidate mask and calculate the percentage pixel lost value
                compare = np.bitwise_and(mask, candidate_mask)
                lost = (mask.sum() - compare.sum()) / mask.sum()  

                if lost >= 0.8:
                    final_mask = np.bitwise_or(final_mask, mask)
                    
        # Finally, add the final combined mask the the annotations dict.
        annotations[image_id][feature] = final_mask.tolist()

<timed exec>:53: RuntimeWarning: invalid value encountered in long_scalars


CPU times: user 6min 8s, sys: 32.4 s, total: 6min 40s
Wall time: 6min 40s


In [3]:
# Save annotations for each image as json files
for k, v in annotations.items():
    file = json.dumps(v, separators=(',', ':'))
    with open('/home/kti01/Documents/My Files/Projects/Overlap/data/ground_truth/annotations_gt/'+k+'.json', 'w') as f:
        json.dump(file, f)